# Identificação de *Motifs*

Em diversas análises anteriores foram identificadas diversas regiões de interesse da proteína, entre as quais motivos.
Estes podem não estar completamente anotados nas bases de dados de referência, sendo por isso um exercício interessante fazer a procura dos mesmos.
Podemos assim validar os conhecidos e destacar outros.

Para este processo foi utilizado o meme suite.

In [16]:
import docker, os

a = %pwd
wd = (a.rsplit('/',2))[0]

GENE = "ORF7a"
BLAST_Results_ncbi=os.path.join("homologue", GENE, "MN908947.3_ORF7a.fasta")
BLAST_Results_uniprot=os.path.join("homologue", GENE, "B20210130A94466D2655679D1FD8953E075198DA8030B5FR.fasta")
REF=os.path.join("reference", GENE, "P0DTC7.fasta")
D_MEME='quay.io/biocontainers/meme:5.3.0--py37pl526h072abfd_0'

WORK_HOST_DIRECTORY=wd
WORK_CONT_DIRECTORY='/opt/project'

In [17]:
client = docker.from_env()
bwa_docker_cont = client.containers.run(
    image=D_MEME,
    volumes={WORK_HOST_DIRECTORY: {'bind': WORK_CONT_DIRECTORY,'mode': 'rw'}},
    working_dir= WORK_CONT_DIRECTORY+'/data/motif/',
    entrypoint="bash",
    command='{0}/scripts_bash/meme.sh {1} {2} {3} {4}'.format(WORK_CONT_DIRECTORY, BLAST_Results_ncbi, BLAST_Results_uniprot, REF, GENE))

In [18]:
from Bio import motifs

with open(os.path.join(wd, "data/motif", GENE, "meme/meme.xml")) as file:
    meme_record = motifs.parse(file, "MEME")

for motif in meme_record:
    print(motif.id, ":" ,motif.name, motif.num_occurrences)

motif_1 : YHYQECVRGTTVLLKEPCPSGTYEGNSPFHPLADNKFALTCTSTHFAFAC 5
motif_2 : ADGTRHTYQLRARSVSPKLFIRQEEVHQE 5
motif_3 : FLIVAALVFIILCFTIKRKTE 5
motif_4 : MKIILFLTLIALASC 5


A utilização do MEME para anotação *de novo* resultou em 4 potências motifs que se encontram em todas as sequências homólogas alinhadas.

:::{figure,myclass}
![Figura 1 - NJ ORF7a](img/weblogo_ORF7a.png)

Weblogos dos motifs anotados *de novo*
:::

:::{figure,myclass}
![Figura 1 - NJ ORF7a](img/meme_loc_ORF7a.png)

Localização dos motifs anotados *de novo*
:::

Com estes motifs mais uma vez coincidem locais identificados anteriormente, por exemplo o dominio [16:81], que sendo uma estrutura tridimensional, pode ser composta por ambos estes motifs de estrutura secundária.

In [19]:
from scripts_python.vcf_gff import gff2rec

motif_in_ref = gff2rec(os.path.join(wd,"data/motif", GENE, "fimo/fimo.gff"))
for seq in motif_in_ref:
    for feature in seq.features:
        print(feature)

type: sequence_motif
location: [17:67](+)
id: YHYQECVRGTTVLLKEPCPSGTYEGNSPFHPLADNKFALTCTSTHFAFAC-MEME-1-1-sp|P0DTC7|NS7A_SARS2
qualifiers:
    Key: Alias, Value: ['MEME-1']
    Key: ID, Value: ['YHYQECVRGTTVLLKEPCPSGTYEGNSPFHPLADNKFALTCTSTHFAFAC-MEME-1-1-sp|P0DTC7|NS7A_SARS2']
    Key: Name, Value: ['YHYQECVRGTTVLLKEPCPSGTYEGNSPFHPLADNKFALTCTSTHFAFAC_sp|P0DTC7|NS7A_SARS2+']
    Key: pvalue, Value: ['3.64e-62']
    Key: qvalue, Value: ['2.62e-60']
    Key: score, Value: ['614']
    Key: sequence, Value: ['YHYQECVRGTTVLLKEPCSSGTYEGNSPFHPLADNKFALTCFSTQFAFAC']
    Key: source, Value: ['fimo']

type: sequence_motif
location: [67:96](+)
id: ADGTRHTYQLRARSVSPKLFIRQEEVHQE-MEME-2-1-sp|P0DTC7|NS7A_SARS2
qualifiers:
    Key: Alias, Value: ['MEME-2']
    Key: ID, Value: ['ADGTRHTYQLRARSVSPKLFIRQEEVHQE-MEME-2-1-sp|P0DTC7|NS7A_SARS2']
    Key: Name, Value: ['ADGTRHTYQLRARSVSPKLFIRQEEVHQE_sp|P0DTC7|NS7A_SARS2+']
    Key: pvalue, Value: ['9.33e-34']
    Key: qvalue, Value: ['8.68e-32']
    Key: score,

Correndo o FIMO sobre a sequência de referência, podemos observar então ver em detalhe a localização e as propriedades destes motifs.

Finalmente corremos o MAST contra as sequências resultantes do BLAST do NCBI (nr/nt). Isto permitiu verificar a conservação deste motifs nas diversas sequências.


:::{figure,myclass}
![Figura 1 - NJ ORF7a](img/mast_loc_ORF7a.png)

Exemplo das localização dos motifs anotados num segundo subset de sequências homólogas.
:::